In [ ]:
# initializing otter-grader
import otter
grader = otter.Notebook()

# Assignment 4: Modeling and Optimization

**Due on June 10 at 11:59 pm**

## Name: 

## Perm number: 

Mathematical modeling is important for describing exactly what the solution will give us. For example, a maximum likelihood estimator (assuming it exists), $\hat\theta$, is a method for finding the parameter that maximies likelihood $L_n$:
$$L_{n}(\hat{\theta} ; x_1, x_2, \dots, x_n)=\max _{\theta \in \Theta} L_{n}(\theta ; x_1, x_2, \dots, x_n)$$
Data $x_1, x_2, \dots$, set of feasible parameters $\Theta$, and likelihood function $L_n$ are given.

Many other applications of optimization exists, and this assignment will give a hands-on introduction to a simple linear programming problem.

In [1]:
import cvxpy as cp
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

# Resource Allocation Problem

You are in charge of a company that makes two hot sauces: $x_1$ liters of Kapatio and $x_2$ liters of Zriracha. We will use optimization technique to find the "best" manufacturing strategy given our resource constraints.

First, we need to define what we mean by "best" strategy. In this scenario, the goal is to obtain the highest revenue possible. While doing so, there are resource constraints we must satisfy. 


For example, in order to manufacture these two hot sauce products, different amount of peppers and vineger are needed. Also, we have only so much total resource available.

Ingridients | Kapatio | Zriracha | Total Available
----------- | ------- | -------- | ------------------
Pepper      | 5       | 7        | 30
Vineger     | 4       | 2        | 12

## Question 1: Resource Constraints

### Question 1.a: Modeling Resource Usage

What is the equation for the amount of pepper needed to manufacture $x_1$ and $x_2$. What is the equation for the amount of vinegar?
(Use [Mathpix](https://mathpix.com/) to write equations)

<!--
BEGIN QUESTION
name: q1a
manual: true
points: 4
-->
<!-- EXPORT TO PDF -->

**YOUR SOLUTION HERE**

### Question 1.b:  Resource Usage vs. Total Resource Constraint

Total amount of pepper needed cannot exceed total available. Write down the inequality expressing this relationship. Do the same for vinegar. These inequalities are your resource constraints.
Additinally, variables $x_1$ and $x_2$ are non-negative: i.e. amount of manufactured goods cannot be negative.

Rewrite the system of constraint inequalities into a matrix inequality: $Ax\leq b$, where $x=(x_1, x_2)^T$. Arrange rows of $A$ and $b$ such that:

* Row 1: total pepper amount constraint
* Row 2: total vinegar amount constraint
* Row 3: Kapatio non-negativity constraint
* Row 4: Zriracha non-negativity constraint

Less than symbol in $Ax\leq b$ means element-wise.

<!--
BEGIN QUESTION
name: q1b1
manual: true
points: 4
-->
<!-- EXPORT TO PDF -->

**YOUR SOLUTION HERE**

Define matrix `A` and vector `b` according to matrix inequality above.

<!--
BEGIN QUESTION
name: q1b2
manual: false
points: 4
-->

In [2]:
A1 = ...
b1 = ...

<!--
BEGIN QUESTION
name: q1b3_hidden
manual: false
points: 4
-->

In [5]:
## leave blank

### Visualizing Feasible Region
In a 2-dimensional plot, we will visualize the area that satisfies both of the resource constraints. Draw $x_1$ on the horizontal axis and $x_2$ on the vertical axis. 

There will be two main components to the plot:
* **Lines** indicating constraint boundaries:  
    e.g. the constraint $x_2\geq 0$ has boundary at $x_2 = 0$.
* **Shaded area** indicating feasible regions:  
    e.g., the whole region $x_2 > 0$ is to be shaded _if_ $x_2\geq 0$ was the only constraint. We will use shading to indicate the region where _all_ constraints are satisfied.

In [8]:
x1_line = np.linspace(-1, 10, 500)
x2_line = np.linspace(-1, 10, 500)

### Question 1.c: Feasible Region Boundary

In a list named `boundary`, create four data frames for each equality in $Ax=b$. These lines indicate where the feasible area ends. Set column names as

* `$x_1$`
* `$x_2$`
* `constraints`

Note the use of latex codes (feel free to use [Mathpix](https://mathpix.com/)). 

#### Toy Example: Drawing Boundaries

Here is a **toy example** of drawing two constraint boundaries by constructing data frames:

In [9]:
z1_line = np.linspace(-1, 10, 500)
z2_line = np.linspace(-1, 10, 500)

boundary = [
    pd.DataFrame({
        '$z_1$': np.ones_like(z2_line)*1,
        '$z_2$': z2_line,
        'constraint': '$z_1\geq 1$'
    }),
    pd.DataFrame({
        '$z_1$': z1_line,
        '$z_2$': np.ones_like(z1_line)*2,
        'constraint': '$z_2\geq 2$'
    }),
]
fig, ax = plt.subplots(figsize=(6, 6))
sns.lineplot(x='$z_1$', y='$z_2$', hue='constraint', data=pd.concat(boundary), ax=ax).axvline(1)
plt.xlim(-1, 6)
plt.ylim(-1, 6)
plt.show()

Sometimes, things just do not work as expected. 

In the toy example code, 
```
sns.lineplot(x='$z_1$', y='$z_2$', hue='constraint', data=pd.concat(boundary), ax=ax).axvline(1)
```
what seems strange about the plotting command? Why was the strange code necessary?

<!--
BEGIN QUESTION
name: q1c1
manual: true
points: 4
-->
<!-- EXPORT TO PDF -->

**YOUR SOLUTION HERE**

#### Example: Resource Constraint Boundary

Now, create a data frame for the non-negativity constraint $x_2\geq 0$ as follows:

In [10]:
pd.DataFrame({'$x_1$':x1_line,               ## x_1 can take on any value
              '$x_2$':x1_line * 0.0,         ## x_2 = 0
              'constraint':'$x_2 \geq 0$'}), ## constraint equation for labeling

Create a list named `boundary` containing four data frames (each corresponding to a constraint). Concatenate data frames in `boundary` to one data frame named `hull`.

<!--
BEGIN QUESTION
name: q1c2
manual: false
points: 6
-->

In [11]:
boundary = [
    ...
    pd.DataFrame({'$x_1$':x1_line,
                  '$x_2$':x1_line * 0.0,
                  'constraint':'$x_2\geq 0$'})
]
hull = ...

### Question 1.d: Interior of Feasible Region

Previous question prepared constraint boundaries, $Ax = b$. In this question, we calculate the interior of the feasible region, which will be shaded in the visualization. First, create a 2-d array of $x_1$ and $x_2$ values. If a point $(x_1, x_2)$ satisfies _every_ constraint, the point will be colored grey.

For example, in order to shade $\{x_1: x_1\geq 1\}\cap\{x_2: x_2\geq 2\}$, we can use the `imshow` method. 

In [16]:
z1_line = np.linspace(-1, 6, 10)
z2_line = np.linspace(-1, 6, 10)
z1_grid, z2_grid = np.meshgrid(z1_line, z2_line)

fig, az = plt.subplots(figsize=(6, 6))
az.imshow(
    ((z1_grid >= 1) & (z2_grid >= 2)).astype(int),
    origin='lower',
    extent=(z1_grid.min(), z1_grid.max(), z2_grid.min(), z2_grid.max()),
    cmap="Greys", alpha=0.3, aspect='equal' 
)
plt.xlim(-1, 6)
plt.ylim(-1, 6)
plt.show()

By dissecting the command below and reading the documentation, report what each of the following lines does:

<!--
BEGIN QUESTION
name: q1d1
manual: true
points: 6
-->
<!-- EXPORT TO PDF -->

* `((y1_grid >= 1) & (y2_grid >= 2)).astype(int)` (What is the output of running this command?)  
    **YOUR SOLUTION HERE**
* `origin='lower'`  
    **YOUR SOLUTION HERE**
* `extent=(y1_grid.min(), y1_grid.max(), y2_grid.min(), y2_grid.max())`  
    **YOUR SOLUTION HERE**
* `cmap='Greys'`  
    **YOUR SOLUTION HERE**
* `alpha=0.3`  
    **YOUR SOLUTION HERE**
* `aspect='equal'`  
    **YOUR SOLUTION HERE**

### Question 1.e: Visualizing the Feasible Region

Finally, create a figure that shows constraint boundaries and the interior region shaded with a light grey color.

Your output will look like this:  
![hull](feasible.png)

<!--
BEGIN QUESTION
name: q1e1
manual: true
points: 8
-->
<!-- EXPORT TO PDF -->

In [17]:
fig, ax = plt.subplots(figsize=(6, 6))
x1_grid, x2_grid = np.meshgrid(x1_line, x2_line)

ax.imshow(
    (
        ...
    ),
    origin='lower',
    extent=(x1_grid.min(), x1_grid.max(), x2_grid.min(), x2_grid.max()),
    cmap="Greys", alpha = 0.3, aspect='equal')


# ax = sns.lineplot(???).axvline(???)
ax = ...

plt.xlim(-1, 6)
plt.ylim(-1, 6)
plt.show()

In the context of linear programming, $Ax\leq b$ is called the _feasible region_ (including the appropriate sections of the boundaries). Denote the (shaded) feasible region as set $C$. Points $(x_1,x_2)\in C$ satisfy all of the constraints.

Describe in plain words the feasible region in the context of hot sauce manufacturing. Specifically, which constraint is violated (if any) by a point at:

<!--
BEGIN QUESTION
name: q1e2
manual: true
points: 6
-->
<!-- EXPORT TO PDF -->

* $(x_1,x_2) = (4, 1)$:  
    **YOUR SOLUTION HERE**
* $(x_1,x_2) = (0, 5)$:  
    **YOUR SOLUTION HERE**
* $(x_1,x_2) = (3, 4)$:  
    **YOUR SOLUTION HERE**

## Question 2: Objective Function

### Question 2.a: Defining Objective Function

Suppose the hot sauces are sold at the same price: \\$5 per liter. 

What is the equation $f(x)$ for the total revenue as a function of $x_1$ and $x_2$? 

The function $f(x)$ is called the objective function.

<!--
BEGIN QUESTION
name: q2a1
manual: true
points: 4
-->
<!-- EXPORT TO PDF -->

**YOUR SOLUTION HERE**

Objective function $f(x)$ is a linear function in $x$. Therefore, $f(x)$ is a 2-dimesional hyperplane. Note that each value of $f(x)$ defines a line in $(x_1, x_2)$ plane.

For example, $f(x)= 0 = c_1x_1 + c_2x_2$ defines a line. A subspace of equal function value is sometimes referred to as a _level set_ or a _contour line_ when visualized.

First, create a numpy array of prices `c` for the two hot sauces, $x_1$ and $x_2$. Then, create a list `f_vals` containing four data frames of contour lines, $f(x) \in \{0, 10, 20, 30\}$. by creating one data frame for each contour line. 

<!--
BEGIN QUESTION
name: q2a2
manual: false
points: 4
-->

In [18]:
c = ...

fig, ax = plt.subplots(figsize=(6, 6))
contours = [
    pd.DataFrame({
        '$x_1$': x1_line,
        '$x_2$': ( 0 - c[0]*x1_line)/c[1],
        '$f(x)$': 0
    }),
    ...
]
f_vals = pd.concat(contours)

# ax = sns.lineplot(???)
ax = ...

plt.xlim(-1, 6)
plt.ylim(-1, 6)
plt.show()

### Question 2.b: Direction of Steepest Increase

Since we want to maximize revenue, we want to increase our objective function as much as possible. Analogous to the minimization example given in a previous lecture, we can repeatedly move in the direction of function increase. In order to determine such direction, compute the gradient of $f(x)$ at $x=(0,0)^T$:
$$\nabla_x f(x) = \begin{pmatrix}\frac{\partial f(x)}{\partial x_1}\\\frac{\partial f(x)}{\partial x_2}\end{pmatrix} $$

<!--
BEGIN QUESTION
name: q2b
manual: true
points: 4
-->
<!-- EXPORT TO PDF -->

**YOUR SOLUTION HERE**

## Question 3: Putting Pieces Together

### Question 3.a: Standard Form of a Linear Programming Problem

Write down the so-called the _standard form_ of a linear programming problem:
$$\begin{aligned}
\max_x & f(x)\\
\text{subject to } & Ax\leq b
\end{aligned}$$

Specifically, write the obejective as an inner product of two vectors: $f(x) = c^T x$, and write the constraint as a vector inequality involving a matrix-vector prduct: $Ax\leq b$, where $A$ is a 4-by-2 matrix.

<!--
BEGIN QUESTION
name: q3a
manual: true
points: 4
-->
<!-- EXPORT TO PDF -->

**YOUR SOLUTION HERE**

### Question 3.b: Computing the Numerical Solution

Therefore, _maximizing_ the revenue is a search over the feasible region for the best point $x^*=(x_1^*, x_2^*)$ that gives the largest revenue. On the otherhand, any _infeasible_ point _not_ in the feasible region cannot be a solution to the constrained optimization problem.

Notationally, the following expression means the same thing:
$$x^* = \arg\max_{\{x: Ax\leq b\}} f(x)$$

Using [CVXPY](https://www.cvxpy.org/), solve for the resource allocation problem with constraints.

<!--
BEGIN QUESTION
name: q3b
manual: false
points: 4
-->

In [23]:
# define variables
x = ...

# define the linear program
problem = cp.Problem(
    ...
)

fstar1 = problem.solve()                                                 # maximum attained function value
xstar1 = pd.DataFrame(x.value.reshape(1, 2), columns=['$x_1$', '$x_2$']) # maximizer x for f

### Question 3.c: Plotting the optimal solution

<!--
BEGIN QUESTION
name: q3c
manual: true
points: 4
-->
<!-- EXPORT TO PDF -->

In [29]:
fig, ax = plt.subplots(figsize=(6, 6))
x1_grid, x2_grid = np.meshgrid(x1_line, x2_line)
ax.imshow(
    (
        (A1[0,0]*x1_grid + A1[0,1]*x2_grid <= b1[0]) & # Pepper constraints
        (A1[1,0]*x1_grid + A1[1,1]*x2_grid <= b1[1]) & # Vinegar constraints
        (A1[2,0]*x1_grid + A1[2,1]*x2_grid <= b1[2]) & 
        (A1[3,0]*x1_grid + A1[3,1]*x2_grid <= b1[3])   # non-negativity constraints 
    ),
    origin='lower',
    extent=(x1_grid.min(), x1_grid.max(), x2_grid.min(), x2_grid.max()),
    cmap="Greys", alpha = 0.3, aspect='equal' 
)
sns.scatterplot(x='$x_1$', y='$x_2$', data=xstar1, ax=ax, s=100)
sns.lineplot(x='$x_1$', y='$x_2$', hue='$f(x)$', data=f_vals, ax=ax)
plt.xlim(-1, 6)
plt.ylim(-1, 6)
plt.show()

## Question 4: Nutrition Problem

During the second world war, the US Army set out to save money without damaging the nutritional health of members of the armed forces. 

> According to [this source](https://ibmdecisionoptimization.github.io/docplex-doc/mp/diet.html), the following problem is a _simple variation of the well-known diet problem that was posed by George Stigler and George Dantzig: how to choose foods that satisfy nutritional requirements while minimizing costs or maximizing satiety._
> 
> _Stigler solved his model "by hand" because technology at the time did not yet support more sophisticated methods. However, in 1947, Jack Laderman, of the US National Bureau of Standards, applied the simplex method (an algorithm that was recently proposed by George Dantzig) to Stigler’s model. Laderman and his team of nine linear programmers, working on desk calculators, showed that Stigler’s heuristic approximation was very close to optimal (only 24 cents per year over the optimum found by the simplex method) and thus demonstrated the practicality of the simplex method on large-scale, real-world problems._
> 
> _The problem that is solved in this example is to minimize the cost of a diet that satisfies certain nutritional constraints._

The file `foods.csv` contains calorie, nutritional content, serving size, and price per serving information about 64 foods. Read it into a data frame named `foods`.

In [30]:
foods = ...
print(foods)

The file `nutritional_constraints.csv` contains healthy nutritional range constraints. Minimum and maximum allowed nutritional contents can be found in this file. Name the variable `requirements`.

In [31]:
requirements = ...
print(requirements)

Extract the nutritional content of foods into a 2-d array named `ncontent`.

In [32]:
ncontent = ...
print(ncontent)

### Question 4.a: Define Constraints

To avoid eating the same foods, limit each food intake to be 2 or less. Also, one cannot consume less than zero servings. Furthermore, apply the nutritional constraints as specified in `nutritional_constraints.csv` (assume that the units are the same as food nutritional contents)

Note that a range constraints, e.g., $2000 \leq \text{total calories} \leq 2250$, can be written as two constraints: $\text{total calories} \leq 2250$ and $-\text{total calories} \leq -2000$. Hence, we can rewrite caloric intake constraints as
$$\begin{aligned}
-(\text{calories in frozen broccoli})x_0 - (\text{calories in raw carrots})x_1 - \dots - (\text{calories in bean bacon soup, w/watr})x_{63} = -c^T x&\leq -2000\\
  \text{calories in frozen broccoli})x_0 + (\text{calories in raw carrots})x_1 + \dots + (\text{calories in bean bacon soup, w/watr})x_{63} = c^T x&\leq 2250\\
\end{aligned}$$
where vector $c$ contains calorie information for all 64 foods and $x$ containts servings consumed of each food. Matrix $U$ and vector $w$ would be such that
$$\begin{aligned}
U=
\begin{pmatrix}
-c^T\\
c^T
\end{pmatrix}
\text{ and }
w = \begin{pmatrix}
-2000\\
2250
\end{pmatrix}
\end{aligned},$$
and the matrix-vector inequality would be $Ux\leq w$. Range constraints of each food can be implemented similarly with identity matrices.

Denote nutritional content information from `foods` data frame as $A$ and denote the `Min` and `Max` columns of `requirements` as vector $b_L$ and $b_U$, respectively. Construct $M$ and $d$ in $M x \leq d$ using $I$ (identity matrix), $A$, $b_L$, $b_U$, and other constants, so that all the range constraints are expressed in $M x\leq d$. (This is a theory question. No coding is involved)

<!--
BEGIN QUESTION
name: q4a
manual: true
points: 4
-->
<!-- EXPORT TO PDF -->

**YOUR SOLUTION HERE**

### Question 4.b: Create Python Variables

Denote the servings of each food as $x_i$ where $i$ is the row index of each food in `foods` data frame: i.e. $x_0$ indicates number of servings of frozen broccoli, $x_1$ indicates that of raw carrots, etc.

* Create cvxpy variable `x` to represent the number of servings of food.
* Create cost vector `a` that gives per serving cost. 
* Create matrix `M` and `d` that lists nutritional content in the following order:
    * Non-negativity constraint of food consumed: i.e. 0 servings or more
    * Upper limit on food consumed: i.e. 2 servings or less
    * Lower limit on consumption of each nutrition: i.e. following `Min` column
    * Upper limit on consumption of each nutrition: i.e. following `Max` column
    
<!--
BEGIN QUESTION
name: q4b
manual: false
points: 4
-->

In [33]:
M = np.concatenate([
    ...
])

d = np.concatenate([
    ...
])

cost = ...

### Question 4.c: Solve the Problem

Use cvxpy to solve for the optimal solution. Use [ECOS as your solver](https://www.cvxpy.org/tutorial/advanced/index.html?highlight=osqp#choosing-a-solver).

<!--
BEGIN QUESTION
name: q4c
manual: false
points: 4
-->

In [38]:
servings = ...

# define the linear program
nutrition_problem = cp.Problem(
    ...
)

# solve linear programming problem with ECOS solver
# https://www.cvxpy.org/tutorial/advanced/index.html?highlight=osqp#choosing-a-solver
fstar2 = ...
xstar2 = servings.value

### Question 4.d: Interpreting the Results

State the results in the context of the problem. How much of each food was consumed? List the foods and their calculated amounts. What is the total cost of feeding one soldier? 

<!--
BEGIN QUESTION
name: q4d
manual: true
points: 4
-->
<!-- EXPORT TO PDF -->

**YOUR SOLUTION HERE**

## Question 5: Revisiting Recommender System

### Question 5.a: Using Linear Algebra for Optimization
In recommender system module, low-rank matrix factorization was used to execute latent factor modeling of movie ratings data.

Specifically, we calculated matrices $U$ and $V$ to solve the following optimization problem (if all ratings were given):
$$
\begin{aligned}
\min_{U,V} f(U,V) &= \min_{U,V} \|R - V U^T\|_F^2
=\min_{U,V} \left\{ \sum_{m=1}^M\sum_{i=1}^I (r_{mi} - v_m u_i^T)^2 \right\}.
\end{aligned}
$$
The best $U$ and $V$ were calculated iteratively by improving on current estimates:
$$
\begin{aligned}
u_i^{\text{new}} &= u_i + 2\alpha(r_{mi} -  v_m u_i^T)\cdot v_m\\
v_m^{\text{new}} &= v_m + 2\alpha(r_{mi} -  v_m u_i^T)\cdot u_i,
\end{aligned}
$$
where $\alpha$ is the step-size that is to be chosen by the user. (We won't discuss the role in this class, but treat it as an arbitrary, but given, parameter) 

We can make calculating the updates more efficient by calculating them with matrix operations. For example, instead of calculating each deviation $\gamma_{mi} = r_{mi} - v_m u_i^T$ separately for all $m=1,2,\dots,M$ and $i=1,2,\dots,I$, matrix $\Gamma$ of all deviations can be computed together using matrix operation _(verify for yourself)_:
$$\Gamma = R - VU^T$$

Similarly, updating $U$ and $V$ can be combined into matrix calculations which makes the optimization procedure more efficient.

First, note that updates for $u_i$, $i=1,2,\dots,I$ can be rewritten as
$$
\begin{aligned}
u_1^{\text{new}} &= u_1 + 2\alpha\gamma_{m1}\cdot v_m\\
u_2^{\text{new}} &= u_2 + 2\alpha\gamma_{m2}\cdot v_m\\
\vdots\quad &\qquad\qquad\vdots\\
u_I^{\text{new}} &= u_I + 2\alpha\gamma_{mI}\cdot v_m.
\end{aligned}
$$
Stacking all $I$ equations into a matrix form, 
$$
\begin{aligned}
U^{\text{new}} &= U + 2\alpha\Gamma_{m-}^T v_m,
\end{aligned}
$$
where $\Gamma_{m-}$ is the $m$-th row of $\Gamma$ (use the notation $\Gamma_{-i}$ for the $i$-th column). 

Note that there are $M$ such update equations (one for each $m=1,2,\dots,M$) that can also be combined into one matrix update equation involving matrices $U$, $V$, $\Gamma$ and scalars. As stated earlier, since $\alpha$ is assumed to be an arbitrary step-size parameter, we can replace $\alpha/M$ with $\alpha$.

Complete the following update equations:
$$
\begin{aligned}
U^{\text{new}} &= U + 2\alpha[\text{some function of }\Gamma][\text{some function of }V]\\
V^{\text{new}} &= V + 2\alpha[\text{some function of }\Gamma][\text{some function of }U]
\end{aligned}
$$

<!--
BEGIN QUESTION
name: q5a
manual: true
points: 4
-->
<!-- EXPORT TO PDF -->

**YOUR SOLUTION HERE**

### Question 5.b: Implementing Updates

Define three functions:

* `update_G(R, U, V)`: computes deviation $R-VU^T$
* `update_U(G, U, V, alpha=0.01)`: calculates update $U^{\text{new}}$
* `update_V(G, U, V, alpha=0.01)`: calculates update $V^{\text{new}}$

Each function should only be one line of matrix operations. Since some elements of `R` are `np.nan` for any missing ratings, `update_U` and `update_V` functions need to be adjusted by using `numpy.nan_to_num` function where appropriate.

<!--
BEGIN QUESTION
name: q5b
manual: false
points: 4
-->

In [43]:
def update_G(R_, U_, V_):
    
    return ...

def update_U(G_, U_, V_, alpha=0.01):
    
    return ...

def update_V(G_, U_, V_, alpha=0.01):
    
    return ...

# small test to help debug (keep intact)
np.random.seed(1)

M_ = 5
I_ = 3
K_ = 2

R_ = np.random.rand(M_, I_).round(1)
R_[0, 0] = R_[3, 2] = np.nan
U_ = np.random.rand(I_, K_).round(1)
V_ = np.random.rand(M_, K_).round(1)
G_ = update_G(R_, U_, V_)

### Question 5.c: Construct Optimization Algorithm

Combine the above functions to implement the optimization algorithm to iteratively compute $U$ and $V$.

But, first, here are functions that will calculate RMSE and quantify the maximum update (in absolute value) made by `update_U` and `update_V` after they are called.

In [47]:
def rmse(X):
    """
    Computes root-mean-square-error, ignoring nan values
    """
    return np.sqrt(np.nanmean(X**2))

def max_update(X, Y, relative=True):
    """
    Compute elementwise maximum update
    
    parameters:
    - X, Y: numpy arrays or vectors
    - relative: [True] compute relative magnitudes
    
    returns
    - maximum difference between X and Y (relative to Y) 
    
    """
    if relative:
        updates = np.nan_to_num((X - Y)/Y)
    else:
        updates = np.nan_to_num(X - Y)
            
    return np.linalg.norm(updates.ravel(), np.inf)

A template for the optimization algorithm is given below. Fill-in the missing portions to complete the algorithm.

<!--
BEGIN QUESTION
name: q5c1
manual: false
points: 4
-->

In [49]:
def compute_UV(Rdf, K=5, alpha=0.01, max_iteration=5000, diff_thr=1e-3):

    R = Rdf.values
    Rone = Rdf.replace(Rdf, 1) # keep data frame metadata

    M, I = R.shape            # number of movies and users
    U = np.random.rand(I, K)  # initialize with random numbers
    V = np.random.rand(M, K)  # initialize with random numbers
    G = update_G(R, U, V)     # calculate residual

    track_rmse = []
    track_update = []
    for i in range(0, max_iteration): 
        
        Unew = update_U(..., ..., ..., ...)
        Gnew = update_G(..., ..., ...)

        Vnew = update_V(..., ..., ..., ...)
        Gnew = update_G(..., ..., ...)

        track_rmse += [{
            'iteration':i, 
            'rmse': rmse(Gnew),
            'max residual change': max_update(Gnew, G, relative=False)
        }]
        track_update += [{
            'iteration':i, 
            'max update':max(max_update(Unew, U), max_update(Vnew, V))
        }]

        U = Unew
        V = Vnew
        G = Gnew
        
        if track_update[-1]['max update'] < diff_thr:
            break
        
    track_rmse = pd.DataFrame(track_rmse)
    track_update = pd.DataFrame(track_update)
    
    kindex = pd.Index(range(0, K), name='k')
    U = pd.DataFrame(U, index=..., columns=...)
    V = pd.DataFrame(V, index=..., columns=...)
    
    return {
        'U':U, 'V':V,
        'rmse': track_rmse,
        'update': track_update
    }
 
Rsmall = pd.read_pickle('ratings_stacked_small.pkl').unstack()

np.random.seed(134) # set seed for tests
output1 = compute_UV(Rsmall, K=10, alpha=0.001)

Running the function on a different sized problem to check if `compute_UV` adapts to changing problem sizes.
There is nothing new to do here

<!--
BEGIN QUESTION
name: q5c2
manual: false
points: 4
-->

In [57]:
# These tests should pass if `compute_UV` works properly
np.random.seed(134) # set seed for tests
output2 = compute_UV(Rsmall.iloc[:7, :5], K=8)

### Question 5.d: Interpret Diagnostic Plots

Following figures tell us if the optimization algorithm is working properly.

In [65]:
import altair as alt
logscale = alt.Scale(type='log', base=10)
fig_rmse = \
    alt.Chart(output1['rmse'])\
    .mark_line()\
    .encode(
        x='iteration:Q', 
        y=alt.Y('rmse:Q', scale=logscale)
    )
fig_max_residual_change = \
    alt.Chart(output1['rmse'])\
    .mark_line()\
    .encode(
        x='iteration:Q', 
        y=alt.Y('max residual change:Q', scale=logscale)
    )
fig_updates = \
    alt.Chart(output1['update'])\
    .mark_line()\
    .encode(
        x='iteration:Q', 
        y=alt.Y('max update:Q', scale=logscale)
    )
alt.vconcat(
    fig_rmse | fig_max_residual_change,
    fig_updates 
)

By referring back to the function used to calculate the quantities in each figure, describe what each figure is showing and interpret the behavior of the optimization algorithm.

<!--
BEGIN QUESTION
name: q5d
manual: true
points: 4
-->
<!-- EXPORT TO PDF -->

**YOUR SOLUTION HERE**

### Question 5.e: Logistic function 

Note the reconstructed ratings can be smaller than 1 and greater than 5. To confine ratings to between the allowed range, we can use the logistic function. Logistic function is defined as 
$$ h(x) = \frac{1}{1+e^{-x}}. $$
It is straightforward to show the derivative is 
$$ h'(x) = \frac{e^{-x}}{1+e^{-x}} = h(x)(1-h(x)). $$
Therefore, we can rescale the ratings from $r_{mi}\in [1, 5]$ to $r_{mi}\in [0, 1]$. Then, we can find the best $U$ and $V$ to optimize the following:
$$ \min_{U,V} \| R - h(VU^T) \|_F^2 = \sum_{m,i} (r_{mi} - h(v_m u_i^T))^2, $$
where function $h$ is applied elementwise.

According to the new objective function, rewrite update functions

<!--
BEGIN QUESTION
name: q5e1
manual: false
points: 4
-->

In [66]:
def logistic(x):
    """
    Evaluates logistic function
    
    """
    return 1/(1+np.exp(-x))

def update_logistic_G(R_, U_, V_):
    
    return ...

def update_logistic_U(G_, U_, V_, alpha=0.01):
    
    logisticVUT = ...              # estimated ratings
    grad = -2 * np.nan_to_num(...) # gradient direction
    return ...                     # gradient descent update from U_

def update_logistic_V(G_, U_, V_, alpha=0.01):
    
    logisticVUT = ...              # estimated ratings
    grad = -2 * np.nan_to_num(...) # gradient direction
    return ...                     # gradient descent update from V_

# small test to help debug (keep intact)
np.random.seed(1)

M_ = 5
I_ = 3
K_ = 2

R_ = np.random.rand(M_, I_).round(1)
R_[0, 0] = R_[3, 2] = np.nan
U_ = np.random.rand(I_, K_).round(1)
V_ = np.random.rand(M_, K_).round(1)
G_ = update_G(R_, U_, V_)

Now create a function `compute_logistic_UV` below:

<!--
BEGIN QUESTION
name: q5e2
manual: false
points: 4
-->

In [70]:
def compute_logistic_UV(Rdf, K=5, alpha=0.01, max_iteration=5000, diff_thr=1e-3):

    R = Rdf.values
    R = (R.copy()-1)/4         # map ratings to between 0 and 1
    Rone = Rdf.replace(Rdf, 1) # keep data frame metadata

    M, I = R.shape                 # number of movies and users
    U = np.random.rand(I, K)-0.5   # initialize with random numbers
    V = np.random.rand(M, K)-0.5   # initialize with random numbers
    G = update_G(R, U, V)          # calculate residual

    track_rmse = []
    track_update = []
    for i in range(0, max_iteration): 
        
        Unew = update_logistic_U(..., ..., ..., ...)
        Gnew = update_logistic_G(..., ..., ...)

        Vnew = update_logistic_V(..., ..., ..., ...)
        Gnew = update_logistic_G(..., ..., ...)

        track_rmse += [{
            'iteration':i, 
            'rmse': rmse(Gnew),
            'max residual change': max_update(Gnew, G, relative=False)
        }]
        track_update += [{
            'iteration':i, 
            'max update':max(max_update(Unew, U), max_update(Vnew, V))
        }]

        U = Unew
        V = Vnew
        G = Gnew
        
        if track_update[-1]['max update'] < diff_thr:
            break
        
    track_rmse = pd.DataFrame(track_rmse)
    track_update = pd.DataFrame(track_update)
    
    kindex = pd.Index(range(0, K), name='k')
    U = pd.DataFrame(U, index=..., columns=...)
    V = pd.DataFrame(V, index=..., columns=...)
    
    return {
        'U':U, 'V':V,
        'rmse': track_rmse,
        'update': track_update
    }

def logistic_rating(U_, V_):
    """
    converts the rating back to 1 to 5 rating
    """
    return( 4*logistic(V_@U_.T) + 1 )
    
np.random.seed(134) # set seed for tests
output3 = compute_logistic_UV(Rsmall, K=10, alpha=0.05)

### Question 5.f: Analyze Large Dataset

Following code will analyze a larger dataset:

In [74]:
# * ratings.pkl: https://ucsb.box.com/shared/static/gi06k2ymeq08bg6253dyrcroepqdmjub.pkl
# * movies.pkl: https://ucsb.box.com/shared/static/9buqthgoy1bbri959zvweim5p7a05hmn.pkl
# * ratings_small.pkl: https://ucsb.box.com/shared/static/qrqvcj2ie1awpr94oxrseshrtufjlquv.pkl
# * users.pkl: https://ucsb.box.com/shared/static/p8rbbinbl2yks9jgv57qd8m4t3u0n5qp.pkl

# run on larger dataset: ratings for 100 movies 
Rbig = pd.read_pickle('ratings_stacked.pkl').unstack().iloc[:100]

np.random.seed(14) # set seed for tests
output4 = compute_logistic_UV(Rbig, K=5, alpha=0.05, max_iteration=500)

Rhatbig = logistic_rating(output4['U'], output4['V'])

In [75]:
fit_vs_obs = pd.concat([
    Rhatbig.rename(columns={'rating':'fit'}),
    Rbig.rename(columns={'rating':'observed'}),
], axis=1).stack().dropna().reset_index()[['fit','observed']]

fit_vs_obs = fit_vs_obs.iloc[np.random.choice(len(fit_vs_obs), 5000)]

alt.Chart(fit_vs_obs).transform_density(
    density='fit',
    bandwidth=0.2,
    groupby=['observed'],
    extent= [1, 5]
).mark_area().encode(
    alt.X('value:Q', ),
    alt.Y('density:Q'),
    alt.Row('observed:N', )
).properties(width=400, height=50)

Consider the above plot. By reading the code, comment on what the plot is illustrating. What happens when you add `counts=True` to `transform_density`? What can you conclude?

<!--
BEGIN QUESTION
name: q5f
manual: true
points: 4
-->
<!-- EXPORT TO PDF -->

**YOUR SOLUTION HERE**

### Question 5.g: Make Recommendation

What movies would you recommend to `user id` 601? Do you see any similarities to movies the user rated high?

<!--
BEGIN QUESTION
name: q5g
manual: true
points: 4
-->
<!-- EXPORT TO PDF -->

**YOUR SOLUTION HERE**

# Running Built-in Tests
1. All tests are in `tests` directory
1. Each python file in `tests` is a test
1. `grader.check('testname')` runs test `'testname'`, e.g. `'q1'`
1. `grader.check_all()` runs all visible tests

In [ ]:
# Run built-in checks
grader.check_all()

In [ ]:
# Uncomment to generate pdf in classic notebook (does not work in JupyterLab):
# import nb2pdf
# nb2pdf.convert('assignment4.ipynb')

# Uncomment to generate pdf using command-line tool:
# ! nb2pdf assignment4.ipynb

# Submission Checklist
1. Check filename is `assignment4.ipynb`
1. Save file to confirm all changes are on disk
1. Run *Kernel > Restart & Run All* to execute all code from top to bottom
1. Check `grader.check_all()` output
1. Save file again to write any new output to disk
1. Check generated pdf that all responses are displayed correctly
1. Submit `assignment4.ipynb` and `assignment4.pdf` to Gradescope